# 2장. 신경망의 수학적 구성요소

## 2.1 신경망과의 첫만남

MNIST 분류문제
- 클래스(class) = 각 범주
- 샘플 (sample) = 데이터 포인트  
- 레이블(lable) = 샘플의 클래스

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131304 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
# 코드 2 - 1 케라스에서 MNIST 데이터셋 적재하기
from keras.datasets import mnist
(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

Using TensorFlow backend.


11493376/11490434 [==============================] - 1s 0us/step


In [0]:
# 코드 2-2 신경망 구조
from keras import models
from keras import layers

network = models.Sequential()
network.add(layers.Dense(512,activation ='relu', input_shape = (28*28,)))
network.add(layers.Dense(10, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.


신경망이 훈련준비를 마치기 위해서는 컴파일 단계에 포함될 3가지가 추가적으로 필요

- 손실함수
- 옵티마이저
- 훈련과 테스트 과정을 모니터링할 지표

In [0]:
# 코드 2 - 3 컴파일 단계
network.compile(optimizer = 'rmsprop', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
# 코드 2 - 4 이미지 데이터 준비하기
train_images = train_images.reshape((60000,28*28))
train_images = train_images.astype('float32')/255 # [0,255] 사이 값인 uint 8 타입으로 데이터 변환

test_images = test_images.reshape((10000,28*28))
test_images = test_images.astype('float32')/255 # [0,255] 사이 값인 uint 8 타입으로 데이터 변환

In [0]:
# 코드 2 - 5 레이블 준비하기(레이블을 범주형으로 인코딩)
from keras.utils import to_categorical

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [0]:
# 코드 :  훈련 실행과 결과
network.fit(train_images,train_labels, epochs = 5, batch_size = 128)
test_loss , test_acc = network.evaluate(test_images,test_labels)
print('test_acc : ', test_acc)

Epoch 1/5
60000/60000 [==============================] - 5s 88us/step - loss: 0.0078 - acc: 0.9980
Epoch 2/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0061 - acc: 0.9982
Epoch 3/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0050 - acc: 0.9985
Epoch 4/5
60000/60000 [==============================] - 5s 86us/step - loss: 0.0036 - acc: 0.9990
Epoch 5/5
10000/10000 [==============================] - 0s 49us/step
test_acc :  0.9827


 **과대적합** 되어 정확도의 차이가 발생
 

---




## 2.2 신경망을 위한 데이터 표현

- 텐서 (tensor) = 데이터를 위한 컨테이너, 즉 임의의 차원 개수를 가지는 행렬, 벡터에 대한 표현
~> 여기서 차원(dimension) 대신 축(axis)이라는 표현을 쓰기도 함.

**2.2.1 스칼라 (0D 텐서)** 

- 스칼라(scala) = 하나의 숫자만 담고있는 텐서.
numpy에서 float32나 float64 데이터 하나가 스칼라 텐더
- 랭크(rank) = 텐서의 축 갯수 (ndim 속성으로 축갯수 확인가능)


**2.2.2 벡터 (1D 텐서)**  : 숫자의 배열



**2.2.3 행렬 (2D 텐서)** : 벡터의 배열



**2.2.4 3D텐서와 고차원 텐서** 
- 3D 텐서 = 행렬의 배열
(보통 0D에서 4D, 동영상의 경우 5D까지)

**2.2.5 핵심 속성**
- 축의 개수(랭크)
- 크기(shape) : 텐서의 각 축을 따라 얼마나 많은 차원이 있는지를 나타낸 파이썬의 튜플 (ex. 2d는 (3,5) )
- 데이터 타입 : 텐서에 포함된 데이터 타입. 사전에 할당되어 연속된 메모리에 저장되어야 하여 가변 길이 문자열을 지원하지 않음 (ndim, shape, dtype으로 확인)

**2.2.6 넘파이로 텐서 조작**
- 슬라이싱 : 파이썬에서 배열에서 특정 원소를 선택하는 것

100001번부터 20001번까지 중간 14*14 의 배열만 슬라이싱하고 싶다면 다음과 같이!

*==> 3차원 직육면체 형태*

In [0]:
# 코드 : numpy slicing
my_slice = train_images[10000:20000, 7:-7, 7:-7]

**2.2.7 배치 데이터**

샘플축(sample axis) = 모든 데이터 텐서의 첫번째 축(인덱스 0)


딥러닝 모델은 한 번에 전체 데이터 셋을 처리하는 것이 아닌 단위를 나누어 처리 
~> 나누는 단위 = <u>배치</u>

In [0]:
# 예시 : train_imgaes에서 크기 128의 첫번째 배치는 다음과 같음
batch = train_images[:128]

**2.2.8 텐서의 실제 사례**

- 벡터데이터 : (samples, features) 크기의 2D 텐서
- 시계열 데이터 또는 시퀀스 데이터 : (sample, timesteps, features) 크기의 3D 텐서
- 이미지 : (sample,frame, width, channels) 또는 (samples, channels, height, width)의 크기의 4D 텐서
- 동영상 : (samples, frames, height, width, channels) 또는 (samples, frame, channels, heigth, width) 크기의 5D 텐서

**2.2.9 벡터 데이터**

하나의 데이터 포인트가 벡터로 인코딩 될 수 있으므로, 배치 데이터는 2D텐서로 인코딩

- 샘플축 = 첫번째 축
- 특성 축 = 두번째 측

**2.2.10 시계열 데이터 또는 시퀀스 데이터**

위의 벡터 데이터에 시간 축이 추가된 데이터 셋에서 사용됨.

- 주식가격 데이터 셋
- 트윗 데이터 셋

**2.2.11 이미지 데이터**

높이, 너비, 컬러 채널으로 한 이미지를 포매팅할 수 있고, 샘플의 갯수가 있어 4D 텐서로 가능. 흑백의 경우, 컬러채널이 1개이기에 3D 텐서로 표현가능

- 채널 마지막 방식 :  텐서플로
- 채널 우선 방식 : 씨아노

**2.2.12 비디오 데이터**

비디오는 이미지를 프레임 단위로 읽는 것이므로 1D 텐서만 추가하면 됨.



---



## 2.3 신경망의 톱니바퀴 : 텐서 연산

**2.3.1원소별 연산**

연산의 결과가 텐서에 있는 원소를 독립적으로 연산함을 의미. 넘파이 배열에서는 시스템에 설치된 BLAS 구현에 복잡한 일들을 위임하며, 이는 병렬화되고 효율적인 텐서 조작 루틴으로 빠른 속도의 연산을 지원함.

relu연산, 덧셈 연산 등이 가능

**2.3.2 브로드캐스팅**

넘파이에서는 브로드 캐스팅을 통해 연산을 진행함.

 - 1단계 : 큰 텐서의 ndim에 맞도록 작은 텐서 축이 추가됨
 - 2단계 : 작은 텐서가 새축을 따라서 큰텐서의 크기에 맞도록 반복됨
 
 알고리즘 수준에서만 연산이 진행됨


 **2.3.3 텐서 점곱**

행렬곱과 비슷

텐서플로에서는 matmul함수를 사용하고 (@ 연산 가능), 넘파이와 케라스에서는 dot연산자를 사용함(dot 함수로도 가능)

고차원의 텐서에는 다음과 같이 연산
- (a,b,c,d) (d,e) -> (a,b,c,e)

**2.3.4 텐서 크기 변환**

텐서 크기의 변환은 특정 크기에 맞게 열과 행을 재배열 하는 것. 전치(transposition) 연산도 다수 실행



In [0]:
# 코드 : 크기 변환
import numpy as np

x = train_images.reshape((60000,28*28))
x = np.transpose(x)

In [0]:
x

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

**2.3.5 텐서 연산의 기하학적 해석**

텐서는 N차원의 좌표 포인트로 해석가능. 그렇기에 벡터 연산도 가능

일반적으로 아핀 변환, 회전, 스케일링 등 기본적인 기하학적 연산은 텐서 연산으로 가능.

**2.3.6 딥러닝의 기하학적 해석**


---




## 2.4 신경망의 엔진 : 그래디언트 기반 최적화

훈련 반복 루프는 다음과 같은 역전파 알고리즘을 통해 반복, 훈련

- 1. 훈련 샘플 x와 상응하는 타깃 y의 배티를 추출
- 2. x를 사용하여 네트워크를 수행하고, 예측값을 구함
- 3. 실제값과 예측값의 차이로 현재 배치의 네트워크에 대한 손실 계산
- 4. 배치에 대한 손실이 감소하도록 네트워크의 가중치를 업데이트

효율적인 가중치 업데이트 방법 = 그래디언트 계산하여 업데이트

**2.4.1 변화율이란?**

결과 값에 대한 함수값의 기울기. 데이터를 아주  작게 변화 시켰을 때 예상되는 데이터 변화값을 도함수로 구함

**2.4.2 텐서 연산의 변화율 : 그래디언트**

다차원 입력인 텐서를 입력 받는 함수에 변화율 개념을 확장.

값에 따라 너무 크게 변화하지 않게 하기 위해, 스케일링 비율인 step 값을 잘 설정해야함

**2.4.3 확률적 경사하강법**

미분가능한 함수가 주어지면 이론적으로 함수의 최솟값을 해석적으로 구할수 있음. ~> 변화율이 0이 되는 지점을 향하여! (모멘텀 이용 시 효율적 계산 가능)

**2.4.3 변화율 연결 : 역전파 알고리즘**
연쇄 법칙(chain rule)을 이용하여 많은 층의 값을 경사하강법으로 연산가능

[ 연쇄 법칙 : f(g(x))′=f′g(x)∗g′(x) ]
